### Number of protests within given distance
Number of protests that happened in the past week within a given ditance. This is using the weekly dataset, where all locations are recorded every week, with an indicator for whether a protest happened here or not

In [2]:
import pandas as pd
import numpy as np

from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [3]:
blm = pd.read_csv('data/blm_weekly_protests.csv')
alt_right = pd.read_csv('data/alt-right_weekly_protests.csv')
distances = pd.read_csv('data/dist_matrix_geoid.csv')
distances = distances.set_index('GEOID10')
urban_pop = pd.read_csv('data/population_data.csv')

In [4]:
def protests_under_dist(data, occ, dist):
    """Creates list containing number of protests
    in the past week under given km.
    Matrix multiplication using the distance matrix 
    and the column recording protest occurrence"""
    
    data = data.copy()
    data[data<=dist] = 1
    data[data>dist] = 0

    protests_under_dist = []

    for i in range(0, 159):
        protests_under_dist.append(np.dot(occ[i], data).tolist())

    protests_under_dist.insert(0, [np.nan for i in range(0, 3601)])
    protests_under_dist = protests_under_dist[0:159]

    return [i for a in protests_under_dist for i in a]

In [5]:
datasets = [blm, alt_right]

for index, data in enumerate(datasets):
    protest_list = data['Protest?'].to_list()
    split_list = [protest_list[x:x+3601] for x in range(0, len(protest_list), 3601)]
    
    # Closest protest in the past week (not same location)
    min_dist = []
    min_geoid = []

    for i in range(0, 159):
        df = split_list[i]*distances
        min_dist.append(df[df!=0].min(axis=1).to_list())
        min_geoid.append(df[df!=0].idxmin(axis=1).to_list())

    min_dist.insert(0, [np.nan for i in range(0, 3601)])
    min_dist = min_dist[0:159]
    flat_min_dist = [i for a in min_dist for i in a] 

    min_geoid.insert(0, [np.nan for i in range(0, 3601)])
    min_geoid = min_geoid[0:159]
    flat_min_geoid = [i for a in min_geoid for i in a] 
    
    # Closest protest in the past week (including same location)
    min_dist = []
    min_geoid = []
    new_list = []

    for item in split_list:
        item = [np.inf if i == 0 else i for i in item]
        new_list.append(item)

    for i in range(0, 159):
        df = new_list[i]*distances
        min_dist.append(df[df!=np.inf].min(axis=1).to_list())
        min_geoid.append(df[df!=np.inf].idxmin(axis=1).to_list())

    min_dist.insert(0, [np.nan for i in range(0, 3601)])
    min_dist = min_dist[0:159]
    flat_min_dist_all = [i for a in min_dist for i in a] 

    min_geoid.insert(0, [np.nan for i in range(0, 3601)])
    min_geoid = min_geoid[0:159]
    flat_min_geoid_all = [i for a in min_geoid for i in a] 
    
    # Total number of protests in the past week
    sum_list = [sum(split_list[i]) for i in range(0, len(split_list))]
    sum_for_df = [val for val in sum_list for _ in range(0, 3601)]
    final_sum = [0 for i in range(0, 3601)] + sum_for_df # need a week lag
    final_sum = final_sum[0:572559]
    
    # Number of protests in the same state in the past week
    data['State'] = data['NAME10'].str.split(',').str[1]
    data['State'] = data['State'].str.strip()
    state_list = data['State'].to_list()
    split_state_list = [state_list[x:x+3601] for x in range(0, len(state_list), 3601)]

    list_lists = []

    for a, b in zip(range(0, 159), range(0, 159)):
        new_list = []
        for c, d in zip(split_list[a], split_state_list[b]):
            if c == 0:
                pass
            elif c == 1:
                new_list.append(d)
        list_lists.append(new_list)

    empty = []
    list_lists.insert(0, empty)
    list_lists = list_lists[0:159]

    dicts = [dict(Counter(list_lists[i])) for i in range(0, 159)]
    protests_per_state = []

    for i, j in zip(range(0, 159), range(0, 159)):
        for a in split_state_list[i]:
            try:
                protests_per_state.append(dicts[j][a])
            except:
                protests_per_state.append(0)
                
    # Time since last event
    time_since = []
    
    for i in range(0, 3601):
        time = []
        weeks = np.nan
        for j in range(0, 159):
            if split_list[j][i] == 0:
                if weeks == np.nan:
                    pass
                else:
                    weeks = weeks + 1

            elif split_list[j][i] == 1:
                weeks = 0

            time.append(weeks)
        time = [np.nan if x == 0 else x for x in time] 
        time_since.append(time)

    last_event = []

    for i in range(0, 159):
        weekly = []
        for j in time_since:
            weekly.append(j[i])

        last_event.append(weekly)

    flat_last_event = [i for a in last_event for i in a] 
    
    # Adding all columns to the df
    data['Protests in past week'] = final_sum
    data['Protests in same state past week'] = protests_per_state
    data['Protests under 500km past week'] = protests_under_dist(distances, split_list, 500)
    data['Protests under 1000km past week'] = protests_under_dist(distances, split_list, 1000)
    data['Protests under 2000km past week'] = protests_under_dist(distances, split_list, 2000)
    data['Distance to closest protest past week'] = flat_min_dist
    data['GEOID closest protest past week'] = flat_min_geoid
    data['Distance to closest protest past week (inc same)'] = flat_min_dist_all
    data['GEOID closest protest past week (inc same)'] = flat_min_geoid_all
    data['Time since last event'] = flat_last_event 
    
    # Population data for closest protest in the past week
    data['GEOID closest protest past week'] = pd.to_numeric(data['GEOID closest protest past week'])
    final_df = data.merge(urban_pop,left_on='GEOID closest protest past week', right_on='GEOID10', how='left')
    columns_format = ['20 to 24 years_x', '20 to 24 years_y', '25 to 34 years_x', '25 to 34 years_y', 
              '65 years and over_x', '65 years and over_y', 'Male_x', 'Male_y',
              'Black or African American_x', 'Black or African American_y',
              'White_x', 'White_y', 'One race_x', 'One race_y', 'Total population_x', 'Total population_y',
                'Sex ratio (males per 100 females)_x', 'Sex ratio (males per 100 females)_y', 
                'Median age (years)_x', 'Median age (years)_y', 'Total housing units_x', 'Total housing units_y']
    for column in columns_format:
        final_df[column] = final_df[column].replace('[%\,]', '', regex=True).astype(float)

    final_df['Dif 20-24'] = final_df['20 to 24 years_y'] - final_df['20 to 24 years_x']
    final_df['Dif 25-34'] = final_df['25 to 34 years_y'] - final_df['25 to 34 years_x']
    final_df['Dif over 65'] = final_df['65 years and over_y'] - final_df['65 years and over_x']
    final_df['Dif Male'] = final_df['Male_y'] - final_df['Male_x']
    final_df['Dif Black'] = final_df['Black or African American_y'] - final_df['Black or African American_x']
    final_df['Dif White'] = final_df['White_y'] - final_df['White_x']
    final_df['Dif one race'] = final_df['One race_x'] - final_df['One race_y']
    final_df['Dif HBCU'] = final_df['HBCU_y'] - final_df['HBCU_x']
    
    # Exporting final dataset
    if index == 0:
        final_df.to_csv('data/blm_final_weekly_data.csv')
        
    elif index == 1:
        final_df.to_csv('data/alt-right_final_weekly_data.csv')

## Merging datasets

Column name suffixes:
- x --> Demographic data for the location and BLM protest data
- y_x --> Data for location of closest BLM protest within the past week 
- y --> Alt-right protest data
- y_y --> Data for location of closest alt-right protest within the past week 

In [6]:
blm_weekly = pd.read_csv('data/blm_final_weekly_data.csv')
right_weekly = pd.read_csv('data/alt-right_final_weekly_data.csv')

columns = blm_weekly.columns.to_list()[0:106]
merged_data = pd.merge(blm_weekly,right_weekly,on=columns, how='left')
merged_data = merged_data.drop(['Unnamed: 0', 'Unnamed: 0_x', 'Unnamed: 0_y_x', 'Unnamed: 0_y_y'], axis = 1)

merged_data.to_csv('data/combined_final_weekly_data.csv')